In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("/content/Watches Bags Accessories.csv",encoding='latin1')
df.head()

,Title,Rating in Stars,Rating Count,Sold Count,Voucher,Delivery,Currency,Current Price,Original Price,Link,Category
0,Yfashion Cartoon Electronic Watch Life Water...,4.6/5,707,6K Sold,0,Free Delivery,Rs.,287,Rs. 520,//www.daraz.pk/products/y-led-3-i258744118-s14...,Kids Watches
1,"100% Imported LED watch for men , boys and Kid...",4.3/5,293,1K Sold,0,0,Rs.,270,Rs. 999,//www.daraz.pk/products/100-led-2023-i40167553...,Kids Watches
2,1 Cartoon Characters Analog Wrist Watch For Ki...,4.5/5,57,452 Sold,0,Free Delivery,Rs.,225,Rs. 750,//www.daraz.pk/products/1-i423737473-s20070599...,Kids Watches
3,M3 Touch LED Bracelet Digital Watch Band Good ...,4.6/5,51,332 Sold,0,Free Delivery,Rs.,160,Rs. 299,//www.daraz.pk/products/m3-led-led-i432456407-...,Kids Watches
4,led Watches For Boys and Girls and kids,4.4/5,54,476 Sold,0,Free Delivery,Rs.,198,Rs. 300,//www.daraz.pk/products/led-i398157365-s192654...,Kids Watches


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1078 entries, 0 to 1077
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Title            1078 non-null   object
 1   Rating in Stars  1078 non-null   object
 2   Rating Count     1078 non-null   int64 
 3   Sold Count       1044 non-null   object
 4   Voucher          1078 non-null   object
 5   Delivery         1078 non-null   object
 6   Currency         1078 non-null   object
 7   Current Price    1078 non-null   object
 8   Original Price   1010 non-null   object
 9   Link             1078 non-null   object
 10  Category         1078 non-null   object
dtypes: int64(1), object(10)
memory usage: 92.8+ KB


In [4]:
def extract_rating(value):
    try:
        return float(value.split('/')[0])
    except ValueError:
        return None

df['Rating in Stars'] = df['Rating in Stars'].apply(extract_rating)
df = df.dropna()

In [5]:
def clean_sales(value):
    value = value.replace(' Sold', '')
    if 'K' in value:
        value = value.replace('K', '')
        value = float(value) * 1000
    return int(value)

df['Sold Count'] = df['Sold Count'].apply(clean_sales)

In [6]:
import re

def extract_numerical_value(string):
    if re.match(r'^\d+ Vouchers$', string):  # Check if string starts with a number followed by ' Vouchers'
        return int(re.match(r'^(\d+)', string).group(1))  # Extract the number
    elif re.match(r'^Rs\.(\d+)', string):  # Check if string starts with 'Rs.'
        return 1  # Extract the number after 'Rs.'
    else:
        return 0

df['Voucher'] = df['Voucher'].apply(extract_numerical_value)

In [7]:
def convert_delivery(value):
    if value == 'Free Delivery':
        return 1
    else:
        return 0

df['Delivery'] = df['Delivery'].apply(convert_delivery)

In [8]:
df['Current Price'] = df['Current Price'].str.replace(',', '').astype(int)

In [9]:
df['Original Price'] = df['Original Price'].str.replace('Rs. ', '').str.replace(',', '').astype(int)

In [10]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Category_encoded'] = label_encoder.fit_transform(df['Category']) + 1

### **Encode Title**

In [ ]:
!pip install -U "tensorflow-text==2.13.*"
!pip install "tf-models-official==2.13.*"

In [12]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

In [13]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [14]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']


In [15]:
def batch_generator(sentences, batch_size):
    for i in range(0, len(sentences), batch_size):
        yield sentences[i:i+batch_size]

batch_size = 50
embeddings = []
i = 0

for batch in batch_generator(df["Title"], batch_size):
    print(i)
    embeddings.append(get_sentence_embeding(batch))
    i = i + 1

embededtext = tf.concat(embeddings, axis=0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [15]:
embededtext = embededtext.numpy()

In [26]:
df['Total Sale'] = df['Current Price'] * df['Sold Count']

In [27]:
df.head()

,Title,Rating in Stars,Rating Count,Sold Count,Voucher,Delivery,Currency,Current Price,Original Price,Link,Category,Category_encoded,Total Sale
0,Yfashion Cartoon Electronic Watch Life Water...,4.6,707,6000,0,1,Rs.,287,520,//www.daraz.pk/products/y-led-3-i258744118-s14...,Kids Watches,1,1722000
1,"100% Imported LED watch for men , boys and Kid...",4.3,293,1000,0,0,Rs.,270,999,//www.daraz.pk/products/100-led-2023-i40167553...,Kids Watches,1,270000
2,1 Cartoon Characters Analog Wrist Watch For Ki...,4.5,57,452,0,1,Rs.,225,750,//www.daraz.pk/products/1-i423737473-s20070599...,Kids Watches,1,101700
3,M3 Touch LED Bracelet Digital Watch Band Good ...,4.6,51,332,0,1,Rs.,160,299,//www.daraz.pk/products/m3-led-led-i432456407-...,Kids Watches,1,53120
4,led Watches For Boys and Girls and kids,4.4,54,476,0,1,Rs.,198,300,//www.daraz.pk/products/led-i398157365-s192654...,Kids Watches,1,94248


###  **Predict Sales Trend**

In [48]:
X_Trends = df.drop(["Link","Title","Currency","Category"], axis=1)
Y_Trends = df["Total Sale"]

In [49]:
X_Trends = np.concatenate((X_Trends,embededtext), axis=1)
X_Trends.shape

(905, 777)

In [50]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [51]:
Scaled_X_Trends = scaler.fit_transform(X)

In [52]:
Scaled_X_Trends.shape

(905, 1545)

In [53]:
Y_Trends.shape

(905,)

In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Scaled_X_Trends, Y_Trends, test_size=0.2, random_state=42)

In [59]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

model = LinearRegression()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error for Total Sale:", mse)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("R-squared (R2) Score:", r2)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

Mean Squared Error for Total Sale: 3.2529767504287634e-11
R-squared (R2) Score: 1.0
Mean Absolute Error (MAE): 7.434494700797355e-07
Root Mean Squared Error (RMSE): 5.7034873107851864e-06


### **Predict the Popularity**

In [63]:
df['Total Ratings'] = df['Rating Count'] * df['Rating in Stars']

In [64]:
X_pop = df.drop(["Link","Title","Currency","Category","Total Sale","Total Ratings"], axis=1)
Y_pop = df["Total Ratings"]

In [61]:
X_pop = np.concatenate((X_pop,embededtext), axis=1)

(905,)

In [62]:
Scaled_X_pop = scaler.fit_transform(X_pop)

In [65]:
Scaled_X_Trends.shape

(905, 1545)

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Scaled_X_pop, Y_pop, test_size=0.2, random_state=42)

In [67]:
model = LinearRegression()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error for Total Sale:", mse)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("R-squared (R2) Score:", r2)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

Mean Squared Error for Total Sale: 1155649533.1803896
R-squared (R2) Score: -492.32718801085514
Mean Absolute Error (MAE): 4160.003766688909
Root Mean Squared Error (RMSE): 33994.845685491644
